In [ ]:
from tensorflow.keras.utils import serialize_keras_object
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import json

In [ ]:
# training period
start = datetime(2020, 1, 1)
end = datetime(2022, 1, 1)

In [ ]:
# Stores all the minute data for our symbol from our training period
qb = QuantBook()
symbol = qb.add_cfd("XAUUSD", Resolution.MINUTE).symbol
history = qb.history(symbol, start, end).loc[symbol]
history.head()

In [ ]:
# Records the percentage change from the previous minute instead of the value
daily_pct_change = history[["open","high","low","close"]].pct_change().dropna()
df = daily_pct_change
df.head()

In [ ]:
# Creates features and labels for the model to be trained on
# Model will consider data from the past half hour to make a decision
n_steps = 30
features = []
labels = []
for i in range(len(df)-n_steps):
    input_data = df.iloc[i:i+n_steps].values
    features.append(input_data)
    
    if df['close'].iloc[i+n_steps] >= 0:
        # Price went up
        label = 1
    else:
        label = 0
    labels.append(label)



In [ ]:
features = np.array(features)
labels = np.array(labels)

In [ ]:
# Split features and labels into training set and test set (70/30)
train_length = int(len(features)*0.7)
X_train = features[:train_length]
X_test = features[train_length:]
y_train = labels[:train_length]
y_test = labels[train_length:]

In [ ]:
# Should be 0.5 (equal proportion of price moving up and price moving down examples)
sum(y_train)/len(y_train)

In [ ]:
test_length = int(len(features)*0.3)
X_train = features[:test_length]
X_test = features[test_length:]
y_train = labels[:test_length]
y_test = labels[test_length:]

In [ ]:
sum(y_test)/len(y_test)

In [ ]:
# Builds an ANN layer by layer (30 -> 20 -> 1)
model = Sequential([Dense(30, input_shape=X_train[0].shape, activation='relu'), Dense(20, activation='relu'), Flatten(), Dense(1, activation='sigmoid')])

In [ ]:
# Configures how the model will learn
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'mse'])

In [ ]:
# Trains the model on the training data
model.fit(X_train, y_train, epochs=5)

In [ ]:
# Gets the models predictions for the test data
y_hat = model.predict(X_test)

In [ ]:
results = pd.DataFrame({'y': y_test.flatten(), 'y_hat':y_hat.flatten()})

In [ ]:
results.plot(title="Model Performance: predicted vs actual")

In [ ]:
pred_train = model.predict(X_train)
scores = model.evaluate(X_train, y_train, verbose = 0)
print("Accuracy on training data: " + str(scores[0]) + "\n Error on training data: " + str(1-scores[0]))

pred_test = model.predict(X_test)
scores = model.evaluate(X_test, y_test, verbose = 0)
print("Accuracy on test data: " + str(scores[1]) + "\n Error on test data: " + str(1-scores[1]))

In [ ]:
# Model representatiion
model_str = json.dumps(serialize_keras_object(model))

In [ ]:
model_key = 'forex_price_predictor'

In [ ]:
# Saves the model so it can be accessed later
qb.ObjectStore.Save(model_key, model_str)

In [ ]:
if qb.ObjectStore.ContainsKey(model_key):
    model_str = qb.ObjectStore.Read(model_key)
    config = json.loads(model_str)['config']
    model = Sequential.from_config(config)

In [ ]:
testDate = datetime.now()

In [ ]:
df = qb.History(symbol, testDate - timedelta(40), testDate).loc[symbol]
df_change = df[["open", "high", "low", "close"]].pct_change().dropna()
model_input = []
for index, row in df_change.tail(30).iterrows():
    model_input.append(np.array(row))
model_input = np.array([model_input])

In [ ]:
if round(model.predict(model_input)[0][0]) == 0:
    print("down")
else:
    print("up")
